In [75]:
from dotenv import load_dotenv
load_dotenv()

True

In [76]:
load_dotenv()

True

In [77]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [78]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [79]:
embeddings.embed_query("hello AI")

[-0.033388201147317886,
 0.034539803862571716,
 0.059474579989910126,
 0.05928615480661392,
 -0.06353538483381271,
 -0.06819586455821991,
 0.08823323249816895,
 0.0344407856464386,
 -0.03278515860438347,
 -0.015814995393157005,
 0.020981667563319206,
 -0.018340280279517174,
 -0.03983214125037193,
 -0.08047076314687729,
 -0.014469259418547153,
 0.03326486423611641,
 0.01425926387310028,
 -0.03404999524354935,
 -0.142915740609169,
 -0.023083265870809555,
 -0.02138010412454605,
 0.0026335634756833315,
 -0.04729275032877922,
 -0.010752726346254349,
 -0.06866799294948578,
 0.03112499974668026,
 0.07594592869281769,
 0.0011283049825578928,
 0.011631982401013374,
 -0.036039214581251144,
 0.04483765363693237,
 0.01839079149067402,
 0.12672799825668335,
 -0.001359735382720828,
 0.008206715807318687,
 0.06909963488578796,
 -0.0807635709643364,
 -0.05841311439871788,
 0.05375451594591141,
 0.026227623224258423,
 -0.00682859867811203,
 -0.05635840445756912,
 0.0032929950393736362,
 -0.072501808404

In [80]:
from sklearn.metrics.pairwise import cosine_similarity

In [81]:
documents=["what is a capital of USA?",
           "Who is a president of USA?",
           "Who is a prime minister of Pakistan?"]

In [82]:
my_query="Narendra modi is prime minister of pakistan?"

In [83]:
document_embedding=embeddings.embed_documents(documents)

In [84]:
document_embedding

[[0.11998698115348816,
  -0.021302608773112297,
  -0.04288088157773018,
  0.06645584106445312,
  -0.0643523558974266,
  -0.04424867779016495,
  0.022408470511436462,
  -0.049873046576976776,
  -0.023437676951289177,
  -0.03397207707166672,
  -0.014048065058887005,
  -0.06065930798649788,
  -0.003906784579157829,
  -0.017782047390937805,
  -0.047971028834581375,
  -0.06668160855770111,
  0.004103216342628002,
  -0.013092760927975178,
  0.04439772665500641,
  0.022350657731294632,
  0.009459580294787884,
  -0.02056453377008438,
  -0.0003356038359925151,
  -0.005685813259333372,
  0.05558698996901512,
  0.02512320503592491,
  -0.002817113883793354,
  0.00875900313258171,
  0.003255280666053295,
  -0.015963423997163773,
  0.014263689517974854,
  -0.11220851540565491,
  0.08968556672334671,
  -0.031083721667528152,
  -0.02422383800148964,
  0.006152089219540358,
  0.08058717846870422,
  0.01824996992945671,
  0.05568312108516693,
  0.01670268550515175,
  0.01589597947895527,
  0.00034103207

In [85]:
query_embedding=embeddings.embed_query(my_query)

In [86]:
len(query_embedding)

384

In [87]:
cosine_similarity([query_embedding],document_embedding)

array([[0.1248396 , 0.32716291, 0.84080423]])

In [88]:
from sklearn.metrics.pairwise import euclidean_distances

In [89]:
euclidean_distances([query_embedding], document_embedding)

array([[1.32299688, 1.16003198, 0.56426194]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [90]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [91]:
index=faiss.IndexFlatL2(384)


In [92]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000209F7965A40> >

In [93]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [94]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['c149ab54-f423-43f1-b965-48aae03068ab',
 'e8a18dde-ba2c-4262-9b94-ea4cfd5cc476',
 '22cf7ef5-425f-4dc2-9c8b-30e059f62d38']

In [95]:
vector_store.index_to_docstore_id

{0: 'c149ab54-f423-43f1-b965-48aae03068ab',
 1: 'e8a18dde-ba2c-4262-9b94-ea4cfd5cc476',
 2: '22cf7ef5-425f-4dc2-9c8b-30e059f62d38'}

In [96]:
results = vector_store.similarity_search("Tell me about AI", k=3)

In [97]:
results

[Document(id='e8a18dde-ba2c-4262-9b94-ea4cfd5cc476', metadata={}, page_content='AI is powerful'),
 Document(id='c149ab54-f423-43f1-b965-48aae03068ab', metadata={}, page_content='AI is future'),
 Document(id='22cf7ef5-425f-4dc2-9c8b-30e059f62d38', metadata={}, page_content='Dogs are cute')]

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [98]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [99]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [100]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
    
)

[]

In [101]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":{"$eq": "tweet"}}
    
)

[]

In [102]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)

In [103]:
results[0].metadata

{}

In [104]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [105]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[]

- inmemory(server)
- ondisk(server)
- cloud(yet to discuss)

In [106]:
vector_store.save_local("today's class faiss index")

In [107]:
new_vector_store=FAISS.load_local(
  "today's class faiss index",embeddings ,allow_dangerous_deserialization=True
)

In [108]:
new_vector_store.similarity_search("langchain")

[]

# Complete RAG

In [109]:
from langchain_community.document_loaders import PyPDFLoader

In [110]:
FILE_PATH = r"llama2.pdf"

In [111]:
loader=PyPDFLoader(FILE_PATH)

In [112]:
len(loader.load())

77

In [113]:
pages=loader.load()

In [114]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [115]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [116]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)

In [117]:
split_docs = splitter.split_documents(pages)

In [118]:
len(split_docs)

615

In [119]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [120]:
vector_store.add_documents(documents=split_docs)

['e17ec9d9-a93a-43f5-9d8e-de8e76a4fd3d',
 'd05f7d60-dab6-46ac-ac35-09c6d8092d6a',
 '616cba8d-0357-4f30-961f-569619f696e1',
 '9babb32a-628e-4ba8-8390-e197d732060e',
 '9bb8cfb3-6122-491a-b253-b6ceeb0f6400',
 'e17f8a97-3923-4220-8d02-93235896c359',
 '8740d027-570f-4850-8e8d-bb78c49829a2',
 '3f526118-8bd7-415c-a455-19f7a43fa45a',
 'b73e6daf-b805-445b-80c1-c49249749d20',
 '87ba16d5-9c13-4250-876a-790a0f6f8124',
 'dff2a02b-533f-4f4d-8373-667996d52783',
 '3a77b09f-cb3b-40e8-9f2d-578835e9348c',
 '63ca8e9d-f3fd-4a0b-88d5-c529c57d7ddb',
 '4c9c6088-11fe-4b55-b143-44497d8a6eb9',
 '703a09b9-cc5a-4c80-a5da-d6c6e028801c',
 '9d13feb2-e060-4eaa-8dba-d4b2bc305069',
 '837a9a69-ca9a-4841-9b3d-25b46a0ab8e8',
 '5048aaee-2d26-4eed-a046-f0cad76b43ad',
 '05546dc1-844c-49e9-a407-96edb88bc549',
 '5207e163-363e-4c14-a41f-6a76defcb806',
 'ff1365f9-702d-4bec-88f9-8e27881c7f32',
 '13910bd3-5f0d-4668-8270-ad2c28efc032',
 'a581ca90-966e-4275-aa77-56dd750026a3',
 '06b15a55-be04-49d8-a51a-7dff210d1b4f',
 'b60be1a5-c777-

In [121]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [122]:
retriever.invoke("what is llama model?")

[Document(id='b60be1a5-c777-44ae-9c13-a1415730150a', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'llama2.pdf', 'total_pages': 77, 'page': 3, 'page_label': '4'}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(id='e79fa299-8325-44ac-a1f6-0ef4d374b3bc', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:3

In [123]:
from dotenv import load_dotenv
load_dotenv()

True

In [132]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-2.0-flash-lite-001')

In [133]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [134]:
import pprint

In [135]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


**[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]**

In [136]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

- context(retriever),prompt(hub),model(google),parser(langchain)

In [138]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [139]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [148]:
rag_chain.invoke("what is llama model?")

'Llama 2 is a new technology with potential risks. It was trained between January 2023 and July 2023. Llama 2 models outperform Llama 1 models.'

# 📋 FAISS Vector Database Tutorial - Complete Code Summary

## 🎯 Overview
This notebook demonstrates a comprehensive implementation of **FAISS (Facebook AI Similarity Search)** vector database using LangChain for similarity search and Retrieval-Augmented Generation (RAG) applications.

---

## 🔧 **Setup & Environment Configuration**

### Environment & API Keys
```python
# Load environment variables and configure tokens
from dotenv import load_dotenv
load_dotenv()
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
```

### Core Libraries Imported
- **LangChain Components**: `HuggingFaceEmbeddings`, `FAISS`, `InMemoryDocstore`
- **FAISS**: Facebook's similarity search library
- **Document Processing**: `PyPDFLoader`, `RecursiveCharacterTextSplitter`
- **ML Metrics**: `cosine_similarity`, `euclidean_distances`
- **LLM Integration**: `ChatGoogleGenerativeAI` (Gemini model)

---

## 🧮 **Embeddings & Similarity Metrics**

### HuggingFace Embeddings Setup
```python
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Creates 384-dimensional embeddings
```

### Similarity Comparison Demonstrated
| Metric | Range | Behavior |
|--------|-------|----------|
| **Cosine Similarity** | [-1, 1] | Focuses on angle only |
| **L2 Distance** | [0, ∞) | Focuses on magnitude + direction |

**Test Query**: "Narendra modi is prime minister of pakistan?"
**Test Documents**: Various political questions about USA and Pakistan

---

## 🏗️ **FAISS Index Types & Recommendations**

### Index Types Comparison
| Feature | Flat | IVF (Inverted File) | HNSW (Graph-based) |
|---------|------|--------------------|--------------------|
| **Search Type** | Exact | Approximate (cluster-based) | Approximate (graph-based) |
| **Speed** | Slow (linear) | Fast (cluster search) | Very Fast (graph walk) |

### Dataset Size Recommendations
| Dataset Size | Recommended Index |
|-------------|-------------------|
| **≤ 100K** | `IndexFlatL2` or `IndexFlatIP` |
| **≤ 1M** | `IndexIVFFlat` or `IndexHNSWFlat` |
| **> 1M** | `IndexIVFPQ` or `IndexHNSWFlat` |

---

## 📊 **Basic FAISS Implementation**

### Vector Store Creation
```python
# L2 Distance Index (384 dimensions)
index = faiss.IndexFlatL2(384)
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
```

### Sample Data & Search
- **Added texts**: ["AI is future", "AI is powerful", "Dogs are cute"]
- **Search query**: "Tell me about AI"
- **Results**: Retrieved top 3 most similar documents

---

## 📄 **Advanced Document Processing**

### Document Creation & Metadata
Created **10 sample documents** with different sources:
- **Sources**: `tweet`, `news`, `website`
- **Content**: Various topics (food, weather, tech, sports, finance)
- **Metadata filtering**: Demonstrated filtering by source type

### Search with Filters
```python
# Filter by source type
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    filter={"source": "tweet"}
)
```

---

## 💾 **Persistence & Storage**

### Three Storage Options Demonstrated
1. **In-Memory** (server-based, temporary)
2. **On-Disk** (server-based, persistent)
3. **Cloud** (mentioned for future discussion)

### Save & Load Operations
```python
# Save locally
vector_store.save_local("today's class faiss index")

# Load from disk
new_vector_store = FAISS.load_local(
    "today's class faiss index", 
    embeddings, 
    allow_dangerous_deserialization=True
)
```

---

## 📖 **PDF Document Processing**

### PDF Loading & Chunking
```python
# Load PDF document
loader = PyPDFLoader("llama2.pdf")
pages = loader.load()

# Split into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Hyperparameter
    chunk_overlap=50     # Hyperparameter
)
split_docs = splitter.split_documents(pages)
```

### Large-Scale Vector Store
- **Document**: LLaMA 2 research paper
- **Chunks created**: Processed into smaller, searchable segments
- **Index used**: `IndexFlatIP` (Inner Product for large documents)

---

## 🤖 **RAG (Retrieval-Augmented Generation) Pipeline**

### Complete RAG Chain Components
1. **Retriever**: Vector store with k=10 similar documents
2. **Prompt**: Hub-pulled RAG prompt template
3. **Model**: Google Gemini 2.0 Flash Lite
4. **Parser**: String output parser

### RAG Chain Implementation
```python
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Query execution
result = rag_chain.invoke("what is llama model?")
```

---

## 🔑 **Key Hyperparameters Used**

| Parameter | Value | Purpose |
|-----------|-------|---------|
| **chunk_size** | 500 | Text splitting size |
| **chunk_overlap** | 50 | Overlap between chunks |
| **k (similarity_search)** | 2-10 | Number of retrieved documents |
| **embedding_dimension** | 384 | HuggingFace model output size |

---

## 🎯 **Key Learning Outcomes**

1. **Vector Embeddings**: Conversion of text to numerical representations
2. **Similarity Search**: Different metrics and their use cases
3. **FAISS Integration**: Efficient similarity search at scale
4. **Document Processing**: PDF loading and intelligent chunking
5. **RAG Implementation**: Complete question-answering pipeline
6. **Persistence**: Saving and loading vector databases
7. **Filtering**: Metadata-based document filtering

---

## 🚀 **Production Considerations**

- **Index Selection**: Choose based on dataset size and speed requirements
- **Chunk Strategy**: Balance between context and specificity
- **Embedding Models**: Consider domain-specific models for better performance
- **Retrieval Parameters**: Tune k-value based on application needs
- **Storage Strategy**: Plan for scalability (in-memory vs. persistent vs. cloud)

---

*This implementation provides a solid foundation for building production-ready similarity search and RAG applications using FAISS and LangChain.*


## 📞 Contact

For questions or suggestions, please open an issue or contact the development team.

## 👨‍💻 About the Author

<div align="center" style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; margin: 20px 0; border-radius: 15px; color: white;">
  <img src="https://img.shields.io/badge/AI%20Engineer-Muhammad%20Atif%20Latif-blue?style=for-the-badge&logo=artificial-intelligence" alt="AI Engineer Badge" style="margin-bottom: 15px;">
  
  <h2 style="margin: 15px 0 10px 0; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; font-weight: 600;">Muhammad Atif Latif</h2>
  
  <p style="font-size: 18px; margin: 10px 0; font-weight: 500; color: #f8f9fa;">
    🚀 Data Scientist & Machine Learning Engineer
  </p>
  
  <p style="font-size: 16px; line-height: 1.6; margin: 20px auto; max-width: 600px; color: #e9ecef;">
    Passionate about transforming data into actionable insights and building intelligent systems that solve real-world challenges. 
    Specialized in end-to-end ML pipeline development, from data preprocessing to production deployment.
  </p>
  
  <div style="margin-top: 20px;">
    <span style="background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 25px; margin: 5px; display: inline-block; font-size: 14px;">🤖 Machine Learning</span>
    <span style="background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 25px; margin: 5px; display: inline-block; font-size: 14px;">🧠 Deep Learning</span>
    <span style="background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 25px; margin: 5px; display: inline-block; font-size: 14px;">📊 Data Analytics</span>
    <span style="background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 25px; margin: 5px; display: inline-block; font-size: 14px;">🌐 MLOps</span>
  </div>
</div>

## 🌐 Connect & Collaborate

<div align="center">

### Let's Build Something Amazing Together!

[![GitHub](https://img.shields.io/badge/GitHub-Follow%20Me-181717?style=for-the-badge&logo=github&logoColor=white)](https://github.com/m-Atif-Latif)
[![LinkedIn](https://img.shields.io/badge/LinkedIn-Connect-0A66C2?style=for-the-badge&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/muhammad-atif-latif-13a171318)
[![Kaggle](https://img.shields.io/badge/Kaggle-Compete-20BEFF?style=for-the-badge&logo=kaggle&logoColor=white)](https://www.kaggle.com/matiflatif)
[![X](https://img.shields.io/badge/X-Follow-000000?style=for-the-badge&logo=x&logoColor=white)](https://x.com/mianatif5867)
[![Instagram](https://img.shields.io/badge/Instagram-Follow-E4405F?style=for-the-badge&logo=instagram&logoColor=white)](https://www.instagram.com/its_atif_ai/)
[![Email](https://img.shields.io/badge/Email-Let's%20Talk-EA4335?style=for-the-badge&logo=gmail&logoColor=white)](mailto:muhammadatiflatif67@gmail.com)
</div>

---

<div align="center" style="margin: 40px 0;">
  <h3 style="color: #2c3e50; margin-bottom: 15px;">📈 My Mission</h3>
  <p style="font-size: 16px; color: #34495e; max-width: 700px; margin: 0 auto; line-height: 1.8;">
    <em>"Bridging the gap between cutting-edge research and practical applications. 
    Every line of code is a step towards a more intelligent and automated future."</em>
  </p>
</div>

<div align="center" style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 30px 0; border: 2px solid #e9ecef;">
  <h4 style="color: #495057; margin-bottom: 15px;">💡 Open for Collaboration</h4>
  <p style="color: #6c757d; font-size: 15px; margin: 10px 0;">
    <strong>Research Projects</strong> • <strong>Industry Applications</strong> • <strong>Open Source Contributions</strong> • <strong>Knowledge Sharing</strong>
  </p>
  <p style="color: #6c757d; font-size: 14px; margin-top: 15px;">
    Feel free to reach out if you have interesting projects, ideas, or just want to discuss the latest in AI/ML!
  </p>
</div>

---

<div align="center" style="padding: 20px 0;">
  <p style="color: #7f8c8d; font-size: 14px; margin: 10px 0;">
    ⭐ <strong>If you found this project helpful, please consider giving it a star!</strong> ⭐
  </p>
  <p style="color: #95a5a6; font-size: 12px; margin-top: 15px;">
    Thank you for exploring this project. Your feedback and contributions are always welcome!
  </p>
</div>

---

<div align="center">
  <img src="https://img.shields.io/badge/Made%20with-❤️%20and%20Python-red?style=for-the-badge" alt="Made with Love">
  <img src="https://img.shields.io/badge/AI%20Powered-Machine%20Learning-brightgreen?style=for-the-badge" alt="AI Powered">
  <img src="https://img.shields.io/badge/Status-Production%20Ready-success?style=for-the-badge" alt="Production Ready">
</div>
